In [ ]:
import os
from os import listdir
from os.path import join,isfile
from tqdm import tqdm
import glob
import copy

import cv2
import numpy as np
from tqdm import tqdm
import tensorflow
import tensorflow as tf
from tensorflow.keras import layers, losses, optimizers

from misc_function import processImage, detail_enhance_lab, recreate_image, PreidictLabel, AdvLoss
from module import DeepGuidedFilter
from my_model import Model

import matplotlib.pyplot as plt
import pickle

In [2]:
def run(image, smooth_image, filename):
    
    # Adv img 폴더
    adv_path =    '../Adv_Img_mobilenet_v2/'
    if not os.path.isdir(adv_path):
        os.makedirs(adv_path)

    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0005)

    with open('../model_weight/model_20200507_9_1.00_0.0088','rb') as w:
        weights = pickle.load(w)

    base_model = tf.keras.applications.MobileNetV2(input_shape=[160, 160, 3], include_top=False,weights='imagenet')

    # white-box model
    model = Model(base_model)
    model.build((None,160,160,3))
    model.set_weights(weights)

    guided_model = DeepGuidedFilter()#
    
    x= processImage(image)    
    gt_smooth = processImage(smooth_image)
    class_x, logit = PreidictLabel(x,model)
    ####
    maxIters = 5000
    
    for it in range(maxIters):
        if not tf.math.equal(int(filename.split('_')[2][0]), class_x):
            break
            
        with tf.GradientTape() as tape:
            #tape.watch(x)     
            x_smooth = guided_model(x,x)# x,gt_smooth
            enh = detail_enhance_lab(x,x_smooth) 
            class_enh, logit_enh = PreidictLabel(tf.expand_dims(enh,axis=0), model)
            
            loss1 = losses.MSE(gt_smooth, x_smooth)
            loss3 = losses.MAE(gt_smooth, x_smooth)
            loss2 = AdvLoss(logit_enh, class_x)
            loss = loss1*1000 + loss2 + loss3*500
            
            #recreat하는 과정에서 뭉개질수있기때문에, 복원한다음의 class값이랑 비교해야함.
            ckeck_enh = recreate_image(enh)
            class_enh,_ = PreidictLabel(np.expand_dims(enh,axis=0),model)
            
            if it % 20 ==0:
                print(f'iter : {it} \t loss:{tf.reduce_mean(loss1)} \t Adv_loss: {loss2}')
            if (class_x != class_enh):
                cv2.imwrite('{}{}'.format(adv_path,filename), recreate_image(enh))
                if(tf.reduce_mean(loss1)<0.0005):
                    break
                         
            variables = guided_model.trainable_variables
            gradients = tape.gradient(loss, variables)
            optimizer.apply_gradients(zip(gradients, variables))

In [ ]:
image=[]
label=[]
filename=[]
for file_name in glob.glob('../Dataset/*.png'):
    filename.append(file_name.split('/')[2])
    label.append(file_name.split('_')[2][0])
    image.append(plt.imread(file_name)[...,:3])
image_set = list(map(lambda x: x, image))
for idx in tqdm(range(0,len(image_set))):
    smooth_image = plt.imread('../Smoothing_Imgs/'+filename[idx])
    run(image_set[idx],smooth_image,filename[idx])